# RIS to CSV converter
Notes
1) The code below can be used to convert RIS files to CSV files
2) The RIS file can be generated using citationchaser.org website


In [5]:
import rispy
import pandas as pd

# Load and parse the RIS file
file_path = 'C:\\Users\\pedro\\Desktop\\Materials World\\references-2024-10-31.ris'
with open(file_path, 'r', encoding='utf-8') as file:
    entries = rispy.load(file)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(entries)
output_csv_path = 'C:\\Users\\pedro\\Desktop\\Materials World\\Review 1.csv'
df.to_csv(output_csv_path, index=False)

print(f"CSV file saved to {output_csv_path}")


CSV file saved to C:\Users\pedro\Desktop\Materials World\Review 1.csv


# How to fetch the reference list with the reference number, doi, title and author (with year)
it uses the crossref API


In [9]:
import requests
import pandas as pd

# Function to fetch references using the CrossRef API
def get_references_from_doi(doi):
    url = f"https://api.crossref.org/works/{doi}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        references = data.get('message', {}).get('reference', [])
        return references
    else:
        print(f"Error fetching data for DOI {doi}: {response.status_code}")
        return None

# Helper function to extract necessary information for the table
def extract_reference_info(ref, order):
    # Extract the title
    title = ref.get('article-title', 'No Title')
    
    # Reference order number
    reference_number = order + 1  # Start from 1
    
    # Extract DOI if available
    ref_doi = ref.get('DOI', 'No DOI')
    
    # Get first author's last name with "et al." if multiple authors
    authors = ref.get('author', [])
    if authors:
        first_author_last_name = authors[0].split(",")[0] + " et al." if len(authors) > 1 else authors[0].split(",")[0]
    else:
        first_author_last_name = "Unknown Author(s)"
    
    # Extract the year of publication
    year = ref.get('year', 'No Year')
    
    # Return a dictionary for easy DataFrame conversion
    return {
        "Title": title,
        "Reference Number": reference_number,
        "DOI": ref_doi,
        "First Author": f"{first_author_last_name} ({year})"
    }

# Example DOI
doi = "10.3390/ma13092078"  # Replace with your DOI
references = get_references_from_doi(doi)

# Create a DataFrame from the extracted references information
if references:
    reference_data = [extract_reference_info(ref, i) for i, ref in enumerate(references)]
    df = pd.DataFrame(reference_data, columns=["Title", "Reference Number", "DOI", "First Author"])
    
    # Display the table
    print(df)
    
    # Optionally, save to CSV
    output_csv_path = 'C:\\Users\\pedro\\Desktop\\Materials World\\test.csv'
    df.to_csv(output_csv_path, index=False)
    print(f"References table saved to {output_csv_path}")
else:
    print("No references found.")


                                                 Title  Reference Number  \
0    Performance improvement of microbial fuel cell...                 1   
1                                             No Title                 2   
2    Conversion of wastes into bioelectricity and c...                 3   
3    High performance platinum group metal-free cat...                 4   
4    Denitrification of water in a microbial fuel c...                 5   
..                                                 ...               ...   
181                                           No Title               182   
182  Electrochemistry and microbiology of microbial...               183   
183  Microbial fuel cell is emerging as a versatile...               184   
184  An overview of electrode materials in microbia...               185   
185  Challenges in the application of microbial fue...               186   

                                 DOI                 First Author  
0         10.1016/j

# How to fetch table information from a PDF

In [ ]:
import pdfplumber

# Specify the path to the PDF file
pdf_path = 'C:\\Users\\pedro\\Desktop\\Materials World\\MFC database\\Review 2.pdf'  # Replace with the path to your downloaded PDF

with pdfplumber.open(pdf_path) as pdf:
    # Loop through the pages
    for i in range(len(pdf.pages)):
        page = pdf.pages[i]
        tables = page.extract_tables()
        for table in tables:
            # Print or process each table
            for row in table:
                print(row)  # Each row is a list of column values
